In [ ]:
from pyannote.audio import Pipeline
from dotenv import load_dotenv
load_dotenv()
import os

> agent consists of theree parts

- Tools: The tools the agent has available to use.
- LLMChain: The LLMChain that produces the text that is parsed in a certain way to determine which action to take.
- The agent class itself: this parses the output of the LLMChain to determin which action to take.

In [ ]:
!pip install langchain==0.0.90

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.chains.conversation.memory import ConversationBufferMemory

In [ ]:
!pip install google-search-results
!pip install google-api-python-client

In [ ]:
# add a custom function to query the balance
import re

def check_balance(word):
    answer= word
    msisdn = [ str(s) for s in re.findall(r'\b\d+\b',  answer ) ]

    if( msisdn == [] ):
        return "Please Input the mobile number to check the account balance ..."
    else:
        return "Number : {} Current Balance is : {}".format( msisdn[0] , 10 )

> google search on the given query

In [ ]:
import os

# initialize the google search wrapper
search = GoogleSerperAPIWrapper()

# define the tool for the agent for querying google
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "Check Account Balance" ,
        func =  lambda number : check_balance( number ) ,
        description = "useful for when you need to get the account balance for the mobile number",
        return_direct=True
        
    )
]

In [ ]:
prefix = """Answer the following questions as best you can, but speaking as a customer call center agent. You have access to the following tools:"""
suffix = """Begin! Remember to speak as a customer call center agent when giving your final answer. 

Question: {input}
{agent_scratchpad}""" # we currently require an agent_scratchpad input variable to put notes on previous actions and observations

In [ ]:
# define the zeroshot agent prompt template
prompt =  ZeroShotAgent.create_prompt(
    tools , # agent usable tools
    prefix = prefix ,
    suffix = suffix ,
    input_variables =  [ "input" , "agent_scratchpad" ]
)

In [ ]:
print( prompt.template )

In [ ]:
# define the llm chain using the openai model
llm_chain =  LLMChain( llm =  OpenAI( temperature = 0 ) , prompt = prompt )

tool_names =  [  i_tool.name  for i_tool in tools ]
# initialize the zeroshot agent
agent =  ZeroShotAgent( llm_chain  = llm_chain , allowed_tools =  tool_names )
# define the agent executor object
agent_runner =  AgentExecutor.from_agent_and_tools( agent = agent , 
                                                    tools = tools , 
                                                    verbose = True  
                                                )

In [ ]:
# we are building a langchain conversational agent
from langchain import OpenAI
from langchain.agents import initialize_agent , Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
import streamlit   as st

In [ ]:
output =  agent_runner.run( "can you suggest me a example tutorial to learn sql ?" )

In [ ]:
output

In [ ]:
output =  agent_runner.run( "10 , can you help me to check my data balance ?" )

In [ ]:
output

## work with multiple inputs

In [ ]:
prefix = """Answer the following questions as best you can. You have access to the following tools:"""
suffix = """When answering, you MUST speak in the following language: {language}.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "language", "agent_scratchpad"]
)

In [ ]:
# define the multi lingual llmchain
multi_lingual_llm = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [ ]:
# define the chat memeory buffer
memory = ConversationBufferMemory( memory_key='chat_history' ) # --> try other memeory types as well

#define the zeroshot agent
multi_lingual_agent = ZeroShotAgent(llm_chain= multi_lingual_llm ,
                                    tools=tools ,
                                    agent= "conversational-react-description" ,
                                    memory  = memory ,
                                    )

# define the agent executor
multi_lingual_executor = AgentExecutor.from_agent_and_tools(agent= multi_lingual_agent  ,
                                                    tools=tools ,
                                                    verbose=True)

In [ ]:
# run the agent on query
output = multi_lingual_executor.run( input="can i check my balance , 0768464525" , 
                                     language = "Sinhala" )

In [ ]:
output

## define the streamlit application

In [ ]:
# initialize the steamlit app
st.header( ":blue[ ChatGPT Agent ] :sunglasses:" ) # short term memory no embedd vector store
user_input =  st.text_input("Customer : ")
# initialize the memory buffer
if "memeory" not in st.session_state :
    st.session_state['memory'] = ""

# add stremalit button
if st.button("Submit"):

    # execute the langchain agent
    st.markdown( multi_lingual_executor.run( input =  user_input  ,
                                             language = "Sinhala"
                                             ) )
    # print the memeory buffer
    # add the conversation history to memeory buffer
    st.session_state['memory'] += memory.buffer
    print( st.session_state['memory'] )